In [ ]:
### Optimization of RBM ###
from __future__ import division, print_function, absolute_import
import os
import sys
sys.path.append('/glade/work/dblaskey/RBM_opt_code/src')
import MOASMO
import Gen_Val_Inputs
import convert_rbm_to_nc
from convert_rbm_to_nc import read_config
import numpy as np
import matplotlib.pyplot as plt
import util
import _pickle as cPickle
import pandas as pd
import sampling
import xarray as xr
import seaborn as sn
import GLP
import gp
import NSGA2
import pickle
import smt
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
import random
import re
from sklearn.metrics import mean_squared_error
import cmocean
import hydroeval as he
import glob
import subprocess
import multiprocessing
import math
import warnings
from scipy.ndimage import gaussian_filter1d

In [7]:
# Calculate air temperature
def mosheni(air_temp, alpha, beta, mu, gamma):
    return mu + (alpha - mu)/(1 + math.exp(gamma*(beta-air_temp)))

# Calibration

In [2]:
#### Begin Code #####
df_sites = pd.read_csv('/glade/u/home/dblaskey/RBM/Regression_Model/Opt_Basins.csv', index_col=0)
df_sites = df_sites[df_sites['type']=="Opt"]
outlets = np.unique(df_sites.outlet_comid.values)

# Read in Observed Data
temp_data = pd.read_csv('/glade/scratch/dblaskey/RBM/temperature_gages.csv', index_col=0)
df = pd.merge(temp_data, df_sites, on="site_no")

In [3]:
psets_df = pd.read_csv('/glade/u/home/dblaskey/RBM/Optimization/Opt_runs_0.csv')
psets_df.rename(columns={ psets_df.columns[0]: "Name" }, inplace = True)

years = range(2013,2018)
start_date = '2013-01-01'
end_date = '2017-12-31'
time_series = pd.date_range(start_date, end_date)

In [10]:
def cal_regression(folder, outlet):
    
    # Calculate air temperature
    def mosheni(air_temp):
        return mu + (alpha - mu)/(1 + math.exp(gamma*(beta-air_temp)))

    print(folder, outlet)
    var_list = psets_df[psets_df['Name'] == folder]

    alpha = var_list.alpha.values[0]
    beta = var_list.beta.values[0]
    mu = var_list.mu.values[0]
    gamma = var_list.gamma.values[0]

    Ordered_reaches_final = pd.read_csv("/glade/scratch/dblaskey/RBM/Input/%s_network.csv"%outlet)
     
    for i, comid in enumerate(df_sites[df_sites['outlet_comid'] == outlet]['COMID'].values):
        cell = Ordered_reaches_final[Ordered_reaches_final['hru'] == comid].node.values[0]
        site_no = df_sites[df_sites['outlet_comid'] == outlet].index.values[i]

        Water = []
        for year in years:
            test = pd.read_csv("/glade/scratch/dblaskey/RBM/RBM_Input/%s_energy_%s"%(outlet,year), sep=" ", header=None, names=["cell", "Tair", "vp", "SW", "LW", "Density","P","Wind"])
            test_id = test[test["cell"] == cell]
            test_id.drop(test_id.tail(1).index, inplace=True)
            test_id['T_smooth'] = 0.1 * test_id['Tair'] + 0.9 * test_id['Tair'].shift()
            test_id['T_smooth']=test_id['T_smooth'].fillna(test_id['Tair'])
            Water.append(test_id.apply(lambda row : mosheni(row['T_smooth']), axis = 1))
            
        Water = np.concatenate(Water)                        
        sim_df = pd.DataFrame(Water, index=time_series, columns=['sim'])
        sim_df.index.name = 'Date'

        # Filter to just one gage for observations
        temp_df = df[df['site_no'] == site_no][['X_00010_00003', 'Date']]
        temp_df = temp_df.rename(columns={"X_00010_00003": "obs"})
        temp_df = temp_df.set_index('Date')
        temp_df.index = pd.to_datetime(temp_df.index)

        # Combine simulation and observation datasets
        df_concat = pd.concat([sim_df, temp_df], axis=1)
        df_concat = df_concat.dropna()
        df_concat = df_concat.loc['2013-10-01':'2017-09-30']
        df_concat = df_concat[df_concat.index.month.isin([5, 6, 7, 8, 9])]
        df_concat = df_concat.reset_index()

        df_concat.to_csv('/glade/scratch/dblaskey/RBM/Regression/Opt/%s/%s.csv' % (folder, comid))

        return [he.evaluator(he.rmse, df_concat.sim.values, df_concat.obs.values)[0],
                np.int(comid),
                folder,
                outlet]

In [ ]:
args_list = []
for folder in np.unique(psets_df.Name.values)[:2]:
    for outlet in outlets[:2]:
        args_list.append((folder, outlet))

#### Preprocess RBM #### 
# set up the multiprocessing pool
pool = multiprocessing.Pool()

# run the function in parallel using the multiprocessing pool
results = pool.starmap(cal_regression, args_list)

# close the multiprocessing pool
pool.close()

pe_basin_0_0000pe_basin_0_0000pe_basin_0_0001pe_basin_0_0001    81000433810000058100000581000433





/glade/u/home/dblaskey/miniconda3/envs/py3/lib/python3.6/site-packages/pandas/core/frame.py:4164: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
# Calculate air temperature
def mosheni(air_temp):
    return mu + (alpha - mu)/(1 + math.exp(gamma*(beta-air_temp)))

RMSE =[]
comid_list =[]
name_list=[]
outlet_list =[]
with warnings.catch_warnings(record=True):
    
    for folder in np.unique(psets_df.Name.values):
        print(folder)
        var_list = psets_df[psets_df['Name'] == folder]

        alpha = var_list.alpha.values[0]
        beta = var_list.beta.values[0]
        mu = var_list.mu.values[0]
        gamma = var_list.gamma.values[0]

        # Build Output Folders
        path = '/glade/scratch/dblaskey/RBM/Regression/Opt/%s/'%folder
        if os.path.exists(path) == False:           
            print("Creating ", folder)
            os.mkdir(path)

        for outlet in outlets:
            Ordered_reaches_final = pd.read_csv("/glade/scratch/dblaskey/RBM/Input/%s_network.csv"%outlet)
            
            for i, comid in enumerate(df_sites[df_sites['outlet_comid'] == outlet]['COMID'].values):
                cell = Ordered_reaches_final[Ordered_reaches_final['hru'] == comid].node.values[0]
                site_no = df_sites[df_sites['outlet_comid'] == outlet].index.values[i]

                Water = []
                for year in years:
                    test = pd.read_csv("/glade/scratch/dblaskey/RBM/RBM_Input/%s_energy_%s"%(outlet,year), sep=" ", header = None,
                    names=["cell", "Tair", "vp", "SW", "LW", "Density","P","Wind"])
                    test_id = test[test["cell"] == cell]
                    test_id.drop(test_id.tail(1).index,inplace=True)
                    test_id['T_smooth'] = 0.1 * test_id['Tair'] + 0.9 * test_id['Tair'].shift()
                    test_id['T_smooth']=test_id['T_smooth'].fillna(test_id['Tair'])
                    Water.append(test_id.apply(lambda row : mosheni(row['T_smooth']), axis = 1))

                Water=np.concatenate(Water)                        
                sim_df = pd.DataFrame(Water, index=time_series, columns=['sim'])
                sim_df.index.name = 'Date'
                sim_df.to_csv('/glade/scratch/dblaskey/RBM/Regression/Opt/%s/%s.csv'%(folder,comid))

                # Filter to just one gage for observations
                temp_df = df[df['site_no']==site_no][['X_00010_00003', 'Date']]
                temp_df = temp_df.rename(columns={"X_00010_00003": "obs"})
                temp_df = temp_df.set_index('Date')
                temp_df.index = pd.to_datetime(temp_df.index)

                # Combine simulation and observation datasets
                df_concat = pd.concat([sim_df,temp_df],axis=1)
                df_concat = df_concat.dropna()
                df_concat = df_concat.loc['2013-10-01':'2017-09-30']
                df_concat = df_concat[df_concat.index.month.isin([5,6,7,8,9])]
                df_concat = df_concat.reset_index()

                RMSE.append(he.evaluator(he.rmse, df_concat.sim.values, df_concat.obs.values)[0])
                comid_list.append(np.int(comid))
                name_list.append(folder)
                outlet_list.append(outlet)
warnings.warn("should not appear")

pe_basin_0_0000
pe_basin_0_0001
pe_basin_0_0002
pe_basin_0_0003
pe_basin_0_0004
pe_basin_0_0005
pe_basin_0_0006
pe_basin_0_0007
pe_basin_0_0008
pe_basin_0_0009
pe_basin_0_0010
pe_basin_0_0011
pe_basin_0_0012
pe_basin_0_0013
pe_basin_0_0014
pe_basin_0_0015
pe_basin_0_0016
pe_basin_0_0017
pe_basin_0_0018
pe_basin_0_0019
pe_basin_0_0020
pe_basin_0_0021
pe_basin_0_0022
pe_basin_0_0023
pe_basin_0_0024
pe_basin_0_0025
pe_basin_0_0026
pe_basin_0_0027
pe_basin_0_0028
pe_basin_0_0029
pe_basin_0_0030
pe_basin_0_0031
pe_basin_0_0032
pe_basin_0_0033
pe_basin_0_0034
pe_basin_0_0035
pe_basin_0_0036
pe_basin_0_0037
pe_basin_0_0038
pe_basin_0_0039
pe_basin_0_0040
pe_basin_0_0041
pe_basin_0_0042
pe_basin_0_0043
pe_basin_0_0044
pe_basin_0_0045
pe_basin_0_0046
pe_basin_0_0047
pe_basin_0_0048
pe_basin_0_0049
pe_basin_0_0050
pe_basin_0_0051
pe_basin_0_0052
pe_basin_0_0053
pe_basin_0_0054
pe_basin_0_0055
pe_basin_0_0056
pe_basin_0_0057
pe_basin_0_0058
pe_basin_0_0059
pe_basin_0_0060
pe_basin_0_0061
pe_basin

In [39]:
# create a dataframe from the results
df_temp = pd.DataFrame({'temp_rmse': RMSE,  'COMID': comid_list, 'Name': name_list, 'Outlet': outlet_list})
df2 = pd.merge(df_temp, df_sites, on=['COMID'])
df2.to_csv('/glade/u/home/dblaskey/RBM/Regression_Model/LHS_results.csv', index=False)

df_LHS = df_temp.groupby('Name').mean("temp_rmse").reset_index().drop(columns=['COMID', 'Outlet'])
df_LHS = df_LHS.merge(psets_df)

df_LHS = df_LHS.drop(["min_velocity", "min_d", "min_w"], axis = 1)

df_LHS.to_csv('/glade/u/home/dblaskey/RBM/Regression_Model/LHS_results_full.csv', index=False)

In [5]:
df_LHS = pd.read_csv('/glade/u/home/dblaskey/RBM/Regression_Model/LHS_results_full.csv')
#d = preprocessing.normalize(df_LHS.drop(["Name", "temp_rmse"], axis = 1),axis=0,return_norm=True)
#normalization_scalar = d[1]

In [8]:
df_LHS[df_LHS['RMSE'] == df_LHS.RMSE.min()]

,Name,RMSE,alpha,beta,mu,gamma,a_d,b_d,a_w,b_w
97,pe_basin_0_0097,1.450777,12.557875,8.1875,2.841925,0.221925,2.164687,0.38125,39.37675,0.190875


In [73]:
param_df = pd.read_csv('/glade/u/home/dblaskey/RBM/Regression_Model/Param_range.csv')
location = "pe_basin"
for i in range(5,10):
    print(i)
    # normalize values
    df_LHS = pd.read_csv('/glade/u/home/dblaskey/RBM/Regression_Model/LHS_results_full_%s.csv'%(i-1))
    
    scaled = df_LHS.drop(["Name", "temp_rmse"], axis = 1)/normalization_scalar

    # start training the surrogate models
    nInput, nOutput = len(param_df), 1
    alpha = 1e-4
    lb = 1e-4
    ub = 1e3

    # perform optimization using the surrogate model
    gen = 100
    crossover_rate = 0.9
    mu = 20
    mum = 20
    N_resample = 20
    leng_lb = 1e-4
    leng_ub = 1e3
    nu = 1.5
    pop = 100

    # start training the surrogate models
    x = scaled.values
    y = df_LHS["temp_rmse"].values

    xlb_single_value_scaled = param_df['Min_Value']/normalization_scalar
    xub_single_value_scaled = param_df['Max_Value']/normalization_scalar

    sm = gp.GPR_Matern(x, y, nInput, nOutput, x.shape[0], xlb_single_value_scaled, xub_single_value_scaled, alpha=alpha, leng_sb=[leng_lb,leng_ub], nu=nu)

    bestx_sm, besty_sm, x_sm, y_sm = \
        NSGA2.optimization(sm, nInput, nOutput, xlb_single_value_scaled.values, xub_single_value_scaled.values, \
                           pop, gen, crossover_rate, mu, mum)
    D = NSGA2.crowding_distance(besty_sm)
    idxr = D.argsort()[::-1][:N_resample]
    x_resample = bestx_sm[idxr,:]
    y_resample = np.zeros((N_resample,nOutput))

    # create test id
    test_id_list=[]
    for id_ in range(N_resample):
        test_id = '%s_%s_%s'%(location, "%i"%i, "%04i"%(id_))
        test_id_list.append(test_id)
    psets_df_new = pd.DataFrame(x_resample, columns=param_df['Var_name'].values, index=test_id_list)

    psets_df_new = psets_df_new*normalization_scalar

    psets_df_new.to_csv('/glade/u/home/dblaskey/RBM/Regression_Model/Opt_runs_%s.csv'%i)
    
    psets_df_new.index=psets_df_new.index.set_names(['Name'])
    psets_df_new = psets_df_new.reset_index()
    
    RMSE =[]
    comid_list =[]
    name_list=[]
    outlet_list =[]
    with warnings.catch_warnings(record=True):

        for folder in np.unique(psets_df_new.Name.values):
            print(folder)
            var_list = psets_df_new[psets_df_new['Name'] == folder]

            alpha = var_list.alpha.values[0]
            beta = var_list.beta.values[0]
            mu = var_list.mu.values[0]
            gamma = var_list.gamma.values[0]

            # Build Output Folders
            path = '/glade/scratch/dblaskey/RBM/Regression/Opt/%s/'%folder
            if os.path.exists(path) == False:           
                print("Creating ", folder)
                os.mkdir(path)

            for outlet in outlets:
                Ordered_reaches_final = pd.read_csv("/glade/scratch/dblaskey/RBM/Input/%s_network.csv"%outlet)
                comid = df_sites[df_sites['outlet_comid'] == outlet]['COMID'].values[0]
                cell = Ordered_reaches_final[Ordered_reaches_final['hru'] == comid].node.values[0]
                site_no = df_sites[df_sites['outlet_comid'] == outlet].index.values[0]

                Water = []
                for year in years:
                    test = pd.read_csv("/glade/scratch/dblaskey/RBM/RBM_Input/%s_energy_%s"%(outlet,year), sep=" ", header = None,
                    names=["cell", "Tair", "vp", "SW", "LW", "Density","P","Wind"])
                    test_id = test[test["cell"] == cell]
                    test_id.drop(test_id.tail(1).index,inplace=True)
                    test_id['T_smooth'] = gaussian_filter1d(test_id['Tair'], sigma=5)

                    Water.append(test_id.apply(lambda row : mosheni(row['T_smooth']), axis = 1))

                Water=np.concatenate(Water)                        
                sim_df = pd.DataFrame(Water, index=time_series, columns=['sim'])
                sim_df.index.name = 'Date'
                sim_df.to_csv('/glade/scratch/dblaskey/RBM/Regression/Opt/%s/%s.csv'%(folder,outlet))

                # Filter to just one gage for observations
                temp_df = df[df['site_no']==site_no][['X_00010_00003', 'Date']]
                temp_df = temp_df.rename(columns={"X_00010_00003": "obs"})
                temp_df = temp_df.set_index('Date')
                temp_df.index = pd.to_datetime(temp_df.index)

                # Combine simulation and observation datasets
                df_concat = pd.concat([sim_df,temp_df],axis=1)
                df_concat = df_concat.dropna()
                df_concat = df_concat.loc['2013-10-01':'2017-09-30']
                df_concat = df_concat[df_concat.index.month.isin([5,6,7,8,9])]
                df_concat = df_concat.reset_index()

                RMSE.append(he.evaluator(he.rmse, df_concat.sim.values, df_concat.obs.values)[0])
                comid_list.append(np.int(comid))
                name_list.append(folder)
                outlet_list.append(outlet)
                        
        df_temp = pd.DataFrame({'temp_rmse': RMSE,  'COMID': comid_list, 'Name': name_list, 'Outlet': outlet_list})
        df2 = pd.merge(df_temp, df_sites, on=['COMID'])
        df2.to_csv('/glade/u/home/dblaskey/RBM/Regression_Model/LHS_results_%s.csv'%i, index=False)

        df_LHS_new = df_temp.groupby('Name').mean("temp_rmse").reset_index().drop(columns=['COMID', 'Outlet'])
        df_LHS_new = df_LHS_new.merge(psets_df_new)
        
        df_LHS_new = pd.concat([df_LHS, df_LHS_new]).reset_index(drop=True)

        df_LHS_new.to_csv('/glade/u/home/dblaskey/RBM/Regression_Model/LHS_results_full_%s.csv'%i, index=False)
        warnings.warn("should not appear")

    print("Done with iteration: ", i)

5
pe_basin_5_0000
Creating  pe_basin_5_0000
pe_basin_5_0001
Creating  pe_basin_5_0001
pe_basin_5_0002
Creating  pe_basin_5_0002
pe_basin_5_0003
Creating  pe_basin_5_0003
pe_basin_5_0004
Creating  pe_basin_5_0004
pe_basin_5_0005
Creating  pe_basin_5_0005
pe_basin_5_0006
Creating  pe_basin_5_0006
pe_basin_5_0007
Creating  pe_basin_5_0007
pe_basin_5_0008
Creating  pe_basin_5_0008
pe_basin_5_0009
Creating  pe_basin_5_0009
pe_basin_5_0010
Creating  pe_basin_5_0010
pe_basin_5_0011
Creating  pe_basin_5_0011
pe_basin_5_0012
Creating  pe_basin_5_0012
pe_basin_5_0013
Creating  pe_basin_5_0013
pe_basin_5_0014
Creating  pe_basin_5_0014
pe_basin_5_0015
Creating  pe_basin_5_0015
pe_basin_5_0016
Creating  pe_basin_5_0016
pe_basin_5_0017
Creating  pe_basin_5_0017
pe_basin_5_0018
Creating  pe_basin_5_0018
pe_basin_5_0019
Creating  pe_basin_5_0019
Done with iteration:  5
6
pe_basin_6_0000
Creating  pe_basin_6_0000
pe_basin_6_0001
Creating  pe_basin_6_0001
pe_basin_6_0002
Creating  pe_basin_6_0002
pe_bas

In [12]:
for local in np.unique(df_sites.Location.values):
    df_temp_loc = df_temp[df_temp['Location']==local]
    
    df_temp_loc = df_temp_loc[['Name', 'temp_rmse']]

    df_LHS_loc = df_temp_loc.groupby('Name').mean("temp_rmse").reset_index()
    df_LHS_loc = df_LHS_loc.merge(psets_df)

    df_LHS_loc = df_LHS_loc.drop(["min_velocity", "min_d", "min_w"], axis = 1)

    df_LHS_loc.to_csv('/glade/u/home/dblaskey/RBM/Regression_Model/LHS_results_full_0-%s.csv'%local, index=False)

In [15]:
### By Location ###
param_df = pd.read_csv('/glade/u/home/dblaskey/RBM/Regression_Model/Param_range.csv')
location = "pe_basin"

for i in range(1,10):
    print(i)
    for local in np.unique(df_sites.Location.values):
        # normalize values
        df_LHS = pd.read_csv('/glade/u/home/dblaskey/RBM/Regression_Model/LHS_results_full_%s-%s.csv'%(i-1, local))

        scaled = df_LHS.drop(["Name", "temp_rmse"], axis = 1)/normalization_scalar

        # start training the surrogate models
        nInput, nOutput = len(param_df), 1
        alpha = 1e-4
        lb = 1e-4
        ub = 1e3

        # perform optimization using the surrogate model
        gen = 100
        crossover_rate = 0.9
        mu = 20
        mum = 20
        N_resample = 20
        leng_lb = 1e-4
        leng_ub = 1e3
        nu = 1.5
        pop = 100

        # start training the surrogate models
        x = scaled.values
        y = df_LHS["temp_rmse"].values

        xlb_single_value_scaled = param_df['Min_Value']/normalization_scalar
        xub_single_value_scaled = param_df['Max_Value']/normalization_scalar

        sm = gp.GPR_Matern(x, y, nInput, nOutput, x.shape[0], xlb_single_value_scaled, xub_single_value_scaled, alpha=alpha, leng_sb=[leng_lb,leng_ub], nu=nu)

        bestx_sm, besty_sm, x_sm, y_sm = \
            NSGA2.optimization(sm, nInput, nOutput, xlb_single_value_scaled.values, xub_single_value_scaled.values, \
                               pop, gen, crossover_rate, mu, mum)
        D = NSGA2.crowding_distance(besty_sm)
        idxr = D.argsort()[::-1][:N_resample]
        x_resample = bestx_sm[idxr,:]
        y_resample = np.zeros((N_resample,nOutput))

        # create test id
        test_id_list=[]
        for id_ in range(N_resample):
            test_id = '%s_%s_%s'%(location, "%i"%i, "%04i"%(id_))
            test_id_list.append(test_id)
        psets_df_new = pd.DataFrame(x_resample, columns=param_df['Var_name'].values, index=test_id_list)

        psets_df_new = psets_df_new*normalization_scalar

        psets_df_new.to_csv('/glade/u/home/dblaskey/RBM/Regression_Model/Opt_runs_%s_%s.csv'%(local,i))

        psets_df_new.index=psets_df_new.index.set_names(['Name'])
        psets_df_new = psets_df_new.reset_index()

        RMSE =[]
        comid_list =[]
        name_list=[]
        outlet_list =[]
        with warnings.catch_warnings(record=True):

            for folder in np.unique(psets_df_new.Name.values):
                print(folder)
                var_list = psets_df_new[psets_df_new['Name'] == folder]

                alpha = var_list.alpha.values[0]
                beta = var_list.beta.values[0]
                mu = var_list.mu.values[0]
                gamma = var_list.gamma.values[0]

                # Build Output Folders
                path = '/glade/scratch/dblaskey/RBM/Regression/Opt/%s/'%folder
                if os.path.exists(path) == False:           
                    print("Creating ", folder)
                    os.mkdir(path)

                for outlet in outlets:
                    Ordered_reaches_final = pd.read_csv("/glade/scratch/dblaskey/RBM/Input/%s_network.csv"%outlet)
                    comid = df_sites[df_sites['outlet_comid'] == outlet]['COMID'].values[0]
                    cell = Ordered_reaches_final[Ordered_reaches_final['hru'] == comid].node.values[0]
                    site_no = df_sites[df_sites['outlet_comid'] == outlet].index.values[0]

                    Water = []
                    for year in years:
                        test = pd.read_csv("/glade/scratch/dblaskey/RBM/RBM_Input/%s_energy_%s"%(outlet,year), sep=" ", header = None,
                        names=["cell", "Tair", "vp", "SW", "LW", "Density","P","Wind"])
                        test_id = test[test["cell"] == cell]
                        test_id.drop(test_id.tail(1).index,inplace=True)
                        test_id['T_smooth'] = gaussian_filter1d(test_id['Tair'], sigma=5)

                        Water.append(test_id.apply(lambda row : mosheni(row['T_smooth']), axis = 1))

                    Water=np.concatenate(Water)                        
                    sim_df = pd.DataFrame(Water, index=time_series, columns=['sim'])
                    sim_df.index.name = 'Date'
                    sim_df.to_csv('/glade/scratch/dblaskey/RBM/Regression/Opt/%s/%s.csv'%(folder,outlet))

                    # Filter to just one gage for observations
                    temp_df = df[df['site_no']==site_no][['X_00010_00003', 'Date']]
                    temp_df = temp_df.rename(columns={"X_00010_00003": "obs"})
                    temp_df = temp_df.set_index('Date')
                    temp_df.index = pd.to_datetime(temp_df.index)

                    # Combine simulation and observation datasets
                    df_concat = pd.concat([sim_df,temp_df],axis=1)
                    df_concat = df_concat.dropna()
                    df_concat = df_concat.loc['2013-10-01':'2017-09-30']
                    df_concat = df_concat[df_concat.index.month.isin([5,6,7,8,9])]
                    df_concat = df_concat.reset_index()

                    RMSE.append(he.evaluator(he.rmse, df_concat.sim.values, df_concat.obs.values)[0])
                    comid_list.append(np.int(comid))
                    name_list.append(folder)
                    outlet_list.append(outlet)

            df_temp = pd.DataFrame({'temp_rmse': RMSE,  'COMID': comid_list, 'Name': name_list, 'Outlet': outlet_list})
            df2 = pd.merge(df_temp, df_sites, on=['COMID'])
            df2.to_csv('/glade/u/home/dblaskey/RBM/Regression_Model/LHS_results_%s-%s.csv'%(i, local), index=False)

            df_LHS_new = df_temp.groupby('Name').mean("temp_rmse").reset_index().drop(columns=['COMID', 'Outlet'])
            df_LHS_new = df_LHS_new.merge(psets_df_new)

            df_LHS_new = pd.concat([df_LHS, df_LHS_new]).reset_index(drop=True)

            df_LHS_new.to_csv('/glade/u/home/dblaskey/RBM/Regression_Model/LHS_results_full_%s-%s.csv'%(i, local), index=False)
            warnings.warn("should not appear")

        print("Done with iteration: ", i)

1
pe_basin_1_0000
pe_basin_1_0001
pe_basin_1_0002
pe_basin_1_0003
pe_basin_1_0004
pe_basin_1_0005
pe_basin_1_0006
pe_basin_1_0007
pe_basin_1_0008
pe_basin_1_0009
pe_basin_1_0010
pe_basin_1_0011
pe_basin_1_0012
pe_basin_1_0013
pe_basin_1_0014
pe_basin_1_0015
pe_basin_1_0016
pe_basin_1_0017
pe_basin_1_0018
pe_basin_1_0019
Done with iteration:  1
pe_basin_1_0000
pe_basin_1_0001
pe_basin_1_0002
pe_basin_1_0003
pe_basin_1_0004
pe_basin_1_0005
pe_basin_1_0006
pe_basin_1_0007
pe_basin_1_0008
pe_basin_1_0009
pe_basin_1_0010
pe_basin_1_0011
pe_basin_1_0012
pe_basin_1_0013
pe_basin_1_0014
pe_basin_1_0015
pe_basin_1_0016
pe_basin_1_0017
pe_basin_1_0018
pe_basin_1_0019
Done with iteration:  1
pe_basin_1_0000
pe_basin_1_0001
pe_basin_1_0002
pe_basin_1_0003
pe_basin_1_0004
pe_basin_1_0005
pe_basin_1_0006
pe_basin_1_0007
pe_basin_1_0008
pe_basin_1_0009
pe_basin_1_0010
pe_basin_1_0011
pe_basin_1_0012
pe_basin_1_0013
pe_basin_1_0014
pe_basin_1_0015
pe_basin_1_0016
pe_basin_1_0017
pe_basin_1_0018
pe_bas

KeyboardInterrupt: 

# Validation 

In [9]:
df_results = pd.read_csv('/glade/u/home/dblaskey/RBM/Regression_Model/LHS_results_full.csv')

In [11]:
psets_df = df_results[df_results['RMSE'] == df_results.RMSE.min()]

In [12]:
#### Begin Code #####
df_sites = pd.read_csv('/glade/u/home/dblaskey/RBM/Validation/Val_Basins.csv', index_col=0)
outlets = np.unique(df_sites.outlet_comid.values)

# Read in Observed Data
temp_data = pd.read_csv('/glade/scratch/dblaskey/RBM/temperature_gages.csv', index_col=0)
df = pd.merge(temp_data, df_sites, on="site_no")

In [13]:
years = range(2017,2022)
start_date = '2017-01-01'
end_date = '2021-09-30'
time_series = pd.date_range(start_date, end_date)

In [ ]:
RMSE =[]
comid_list =[]
outlet_list =[]
with warnings.catch_warnings(record=True):
    
    alpha = psets_df.alpha.values[0]
    beta = psets_df.beta.values[0]
    mu = psets_df.mu.values[0]
    gamma = psets_df.gamma.values[0]

    for outlet in outlets:
        Ordered_reaches_final = pd.read_csv("/glade/scratch/dblaskey/RBM/Input/%s_network.csv"%outlet)
        comids = df_sites[df_sites['outlet_comid'] == outlet]['COMID'].values
        site_nos = df_sites[df_sites['outlet_comid'] == outlet].index.values
        
        for i,comid in enumerate(comids):
            print(comid)
            site_no = site_nos[i]
            cell = Ordered_reaches_final[Ordered_reaches_final['hru'] == comid].node.values[0]
            
            Water = []
            for year in years:
                test = pd.read_csv("/glade/scratch/dblaskey/RBM/RBM_Input/%s_energy_%s"%(outlet,year), sep=" ", header = None,
                names=["cell", "Tair", "vp", "SW", "LW", "Density","P","Wind"])
                test_id = test[test["cell"] == cell]
                test_id.drop(test_id.tail(1).index,inplace=True)
                test_id['T_smooth'] = 0.1 * test_id['Tair'] + 0.9 * test_id['Tair'].shift()
                test_id['T_smooth']=test_id['T_smooth'].fillna(test_id['Tair'])

                Water.append(test_id.apply(lambda row : mosheni(row['T_smooth']), axis = 1))

            Water=np.concatenate(Water)                        
            sim_df = pd.DataFrame(Water, index=time_series, columns=['sim'])
            sim_df.index.name = 'Date'
            sim_df.to_csv('/glade/scratch/dblaskey/RBM/Regression/Val/%s.csv'%(comid))

            # Filter to just one gage for observations
            temp_df = df[df['site_no']==site_no][['X_00010_00003', 'Date']]
            temp_df = temp_df.rename(columns={"X_00010_00003": "obs"})
            temp_df = temp_df.set_index('Date')
            temp_df.index = pd.to_datetime(temp_df.index)

            # Combine simulation and observation datasets
            df_concat = pd.concat([sim_df,temp_df],axis=1)
            df_concat = df_concat.dropna()
            df_concat = df_concat.loc['2017-10-01':'2021-09-30']
            df_concat = df_concat[df_concat.index.month.isin([5,6,7,8,9])]
            df_concat = df_concat.reset_index()

            RMSE.append(he.evaluator(he.rmse, df_concat.sim.values, df_concat.obs.values)[0])
            comid_list.append(np.int(comid))
            outlet_list.append(outlet)
    warnings.warn("should not appear")

In [74]:
# create a dataframe from the results
df_temp = pd.DataFrame({'temp_rmse': RMSE,  'COMID': comid_list, 'Outlet': outlet_list})
df2 = pd.merge(df_temp, df_sites, on=['COMID'])
df2.to_csv('/glade/u/home/dblaskey/RBM/Regression_Model/val_results.csv', index=False)

In [75]:
df2.temp_rmse.mean()

2.8698513461813593

In [76]:
df2.groupby('Location').temp_rmse.mean()

Location
Interior    2.897326
North       4.092079
South       2.314640
Name: temp_rmse, dtype: float64

In [77]:
df2.groupby('type').temp_rmse.mean()

type
OB    3.104329
OG    2.497781
VB    3.137267
VG    2.889884
Name: temp_rmse, dtype: float64